In [1]:
pip install datasets


Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import list_datasets, load_dataset, list_metrics,load_metric
# Print all the available datasets
print(len(list_datasets()))

C:\Users\catch\AppData\Local\Temp\ipykernel_17628\437539306.py:3: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  print(len(list_datasets()))


67738


In [2]:
dataset = load_dataset('imdb')


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
dataset['train'][2]

{'text': "If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />",
 'label': 0}

In [5]:
dataset['train'].description

'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

In [6]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [7]:
from transformers import AutoTokenizer
brt_tkn = AutoTokenizer.from_pretrained("bert-base-cased")
def generate_tokens_for_imdb(examples):
    return brt_tkn(examples["text"], padding="max_length",truncation=True)
tkn_datasets = dataset.map(generate_tokens_for_imdb,batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
training_dataset = tkn_datasets["train"].shuffle(seed=42).select(range(200))
evaluation_dataset = tkn_datasets["test"].shuffle(seed=42).select(range(200))

In [18]:
from huggingface_hub import login
login()

In [9]:
from transformers import AutoModelForSequenceClassification
mdl = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
pip install accelerate -U

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/258.1 kB ? eta -:--:--
   ------------------------------ --------- 194.6/258.1 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 258.1/258.1 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="imdb")

In [11]:
import numpy as np
from datasets import load_metric
mdl_metrics = load_metric("accuracy")
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return mdl_metrics.compute(predictions=predictions,references=labels)

C:\Users\catch\AppData\Local\Temp\ipykernel_17628\3807583209.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  mdl_metrics = load_metric("accuracy")


In [12]:
from transformers import TrainingArguments, Trainer
trng_args = TrainingArguments(output_dir="test_trainer",evaluation_strategy="epoch", num_train_epochs=3)


In [15]:
Mdl_trainer = Trainer(
 model=mdl,
 args=trng_args,
 train_dataset=training_dataset,
 eval_dataset=evaluation_dataset,
 compute_metrics=calculate_metrics)

In [17]:
Mdl_trainer.train()

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.6853758096694946, 'eval_accuracy': 0.525, 'eval_runtime': 170.6622, 'eval_samples_per_second': 1.172, 'eval_steps_per_second': 0.146, 'epoch': 1.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.6177318692207336, 'eval_accuracy': 0.735, 'eval_runtime': 167.295, 'eval_samples_per_second': 1.195, 'eval_steps_per_second': 0.149, 'epoch': 2.0}


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.5452054738998413, 'eval_accuracy': 0.75, 'eval_runtime': 169.3374, 'eval_samples_per_second': 1.181, 'eval_steps_per_second': 0.148, 'epoch': 3.0}
{'train_runtime': 1757.2924, 'train_samples_per_second': 0.341, 'train_steps_per_second': 0.043, 'train_loss': 0.6265591939290365, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.6265591939290365, metrics={'train_runtime': 1757.2924, 'train_samples_per_second': 0.341, 'train_steps_per_second': 0.043, 'train_loss': 0.6265591939290365, 'epoch': 3.0})

In [19]:
Mdl_trainer.save_model()

In [22]:
metrics = Mdl_trainer.evaluate(evaluation_dataset)
Mdl_trainer.log_metrics("eval", metrics)
Mdl_trainer.save_metrics("eval", metrics)

  0%|          | 0/25 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =       0.75
  eval_loss               =     0.5452
  eval_runtime            = 0:02:54.48
  eval_samples_per_second =      1.146
  eval_steps_per_second   =      0.143
